<a href="https://colab.research.google.com/github/Samin-Sadaf7/Image_works/blob/main/Finetune_EfficientDet_for_Traffic_Sign_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparations

## Download the dataset

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import os
import sys

%cd /content

/content


## Download the code

In [16]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 97475, done.
remote: Counting objects: 100% (755/755), done.
remote: Compressing objects: 100% (350/350), done.
remote: Total 97475 (delta 462), reused 623 (delta 398), pack-reused 96720
Receiving objects: 100% (97475/97475), 613.70 MiB | 26.57 MiB/s, done.
Resolving deltas: 100% (70880/70880), done.


In [3]:
if not os.path.isdir("automl"):
    # clone the repository
    !git clone --depth 1 https://github.com/google/automl

    # checkout to the latest commit that we used when we're creating this notebook
    %cd automl
    !git checkout 1ec78d22aa9f8b7d33b9cf3a177e05dcc6b4a093

    # change the working directory
    %cd efficientdet

    # install required packages
    # if it fails to install pycocotools, please manually remove pycocotools from requirements.txt and run again
    %pip install -r requirements.txt
    %pip install -U "git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI"

PROJ_DIR = os.path.join("/content")

Cloning into 'automl'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 181 (delta 16), reused 71 (delta 0), pack-reused 0
Receiving objects: 100% (181/181), 13.79 MiB | 18.83 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/automl
fatal: reference is not a tree: 1ec78d22aa9f8b7d33b9cf3a177e05dcc6b4a093
/content/automl/efficientdet
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-8eg43p0l
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-8eg43p0l
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-8zbrxx50
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-8zbrxx50
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done


## Download the pretrained model

In [4]:
MODEL = "efficientdet-d1"
if not os.path.exists(f"{MODEL}.tar.gz"):
    !curl -O https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco2/{MODEL}.tar.gz
    !tar xvzf {MODEL}.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 46.5M  100 46.5M    0     0  18.2M      0  0:00:02  0:00:02 --:--:-- 18.2M
efficientdet-d1/
efficientdet-d1/model.meta
efficientdet-d1/model.index
efficientdet-d1/checkpoint
efficientdet-d1/d1_coco_val.txt
efficientdet-d1/d1_coco_test-dev2017.txt
efficientdet-d1/model.data-00000-of-00001


In [5]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 97475, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 97475 (delta 21), reused 32 (delta 16), pack-reused 97433
Receiving objects: 100% (97475/97475), 613.51 MiB | 23.13 MiB/s, done.
Resolving deltas: 100% (70962/70962), done.


In [12]:
%cd models/research
# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
%cp object_detection/packages/tf2/setup.py .
!pip install --use-feature=2020-resolver .

/content
/content/models/research

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

option --use-feature: invalid choice: '2020-resolver' (choose from 'fast-deps', 'truststore', 'no-binary-enable-wheel-cache')


# Preprocess the dataset

## Preprocessing parameters

In [5]:
labels = [
    'Crossroads',
    'no use of horn',
    'Road Hump',
    'Narrow Bridge',
    'speed limit 40 km/h',
    'Side Road Right',
    'School',
    'Sharp Bend To The Left',
    'speed limit 80 km/h',
    'Side Road Left',
    'PEDESTRIAN CROSSING',
    'No Overtaking',
    'Sharp Bend To The Right',
    'Speed Limit 60 km/h',
    'U Turn'
]

with open('label_map.pbtxt', 'w') as f:
    for i, label in enumerate(labels, 1):
        f.write('item {\n')
        f.write('\tid: {}\n'.format(i))
        f.write('\tname: "{}"\n'.format(label))
        f.write('}\n')

In [11]:
with open('classnames.txt','w') as f:
  for label in labels:
      f.write(label)
      f.write(',')

In [4]:
!python /content/create_tfrecord.py --data_dir='/path/to/train' --output_path='/path/to/output/train.record' --label_map_path='/path/to/label_map.pbtxt' --classes_file='/path/to/classnames.txt'

2024-07-15 06:57:09.833617: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-15 06:57:09.833695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-15 06:57:09.835760: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-15 06:57:09.848462: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 06:57:11.749772: W tensorflow/compiler/tf2

In [ ]:
PROJ_DIR = os.path.join("/content/gdrive/MyDrive/Dataset- -Conference")

In [ ]:
DATASET_DIR = os.path.join(PROJ_DIR)
TFRECORD_DIR = os.path.join(PROJ_DIR, "tfrecords")

TRAIN_SET_FILE = os.path.join(DATASET_DIR, "train")
TRAIN_SET_FILE = "/content/gdrive/MyDrive/Dataset- -Conference/train/train.txt"
VAL_SET_FILE = os.path.join(DATASET_DIR, "val")
TEST_SET_FILE = os.path.join(DATASET_DIR,  "test")

SAMPLES_PER_FILE = 10000   # we only construct a subset, in total there are 100,000 samples in each set

LP_CLASS = 1   # class id for the license plate

In [ ]:
import re
import hashlib

import PIL.Image
import tensorflow as tf
from tensorflow.compat.v1.python_io import tf_record_iterator


def filename_to_annotation(filename):
    """
    Annotations are embedded in file name.

    A sample image name is "025-95_113-154&383_386&473-386&473_177&454_154&383_363&402-0_0_22_27_27_33_16-37-15.jpg". Each name can be splited into seven fields. Those fields are explained as follows.

        Area: Area ratio of license plate area to the entire picture area.

        Tilt degree: Horizontal tilt degree and vertical tilt degree.

        Bounding box coordinates: The coordinates of the left-up and the right-bottom vertices.

        Four vertices locations: The exact (x, y) coordinates of the four vertices of LP in the whole image. These coordinates start from the right-bottom vertex.

        License plate number: Each image in CCPD has only one LP. Each LP number is comprised of a Chinese character, a letter, and five letters or numbers. A valid Chinese license plate consists of seven characters: province (1 character), alphabets (1 character), alphabets+digits (5 characters). "0_0_22_27_27_33_16" is the index of each character. These three arrays are defined as follows. The last character of each array is letter O rather than a digit 0. We use O as a sign of "no character" because there is no O in Chinese license plate characters.

        Brightness: The brightness of the license plate region.

        Blurriness: The Blurriness of the license plate region.
    """

    m = re.match(
        r'^'
        r'([0-9]+)-'
        r'([0-9]+)_([0-9]+)-'
        r'([0-9]+)&([0-9]+)_([0-9]+)&([0-9]+)-'
        r'([0-9]+)&([0-9]+)_([0-9]+)&([0-9]+)_([0-9]+)&([0-9]+)_([0-9]+)&([0-9]+)-'
        r'([0-9_]+)-'
        r'([0-9]+)-'
        r'([0-9]+)'
        r'.png$'
        , filename
    )

    if m:
        m = m.groups()
        anno = {
            "area_ratio": int(m[0]),
            "tilt_h": int(m[1]),
            "tile_v": int(m[2]),
            "bbox": {
                "left": int(m[3]),
                "top": int(m[4]),
                "right": int(m[5]),
                "bottom": int(m[6])
            },
            "lp": str(m[15]),
            "brightness": int(m[16]),
            "blurriness": int(m[17])
        }
        assert anno["bbox"]["left"] < anno["bbox"]["right"]
        assert anno["bbox"]["top"] < anno["bbox"]["bottom"]
    else:
        raise ValueError("the filename cannot be resolved")

    return anno


def create_example(id, filepath, annotation):
    img_raw = open(filepath, "rb").read()
    key = hashlib.sha256(img_raw).hexdigest()
    filename = os.path.basename(filepath)
    width, height = PIL.Image.open(filepath).size

    example = tf.train.Example(features=tf.train.Features(feature={
        "image/height": tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        "image/width": tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        "image/filename": tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode("utf-8")])),
        "image/source_id": tf.train.Feature(bytes_list=tf.train.BytesList(value=[str(id).encode("utf-8")])),
        "image/key/sha256": tf.train.Feature(bytes_list=tf.train.BytesList(value=[key.encode("utf-8")])),
        "image/encoded": tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
        "image/format": tf.train.Feature(bytes_list=tf.train.BytesList(value=["jpg".encode("utf8")])),
        "image/object/bbox/xmin": tf.train.Feature(float_list=tf.train.FloatList(value=[annotation["bbox"]["left"] / width])),
        "image/object/bbox/xmax": tf.train.Feature(float_list=tf.train.FloatList(value=[annotation["bbox"]["right"] / width])),
        "image/object/bbox/ymin": tf.train.Feature(float_list=tf.train.FloatList(value=[annotation["bbox"]["top"] / height])),
        "image/object/bbox/ymax": tf.train.Feature(float_list=tf.train.FloatList(value=[annotation["bbox"]["bottom"] / height])),
        "image/object/class/text": tf.train.Feature(bytes_list=tf.train.BytesList(value=["license_plate".encode("utf-8")])),
        "image/object/class/label": tf.train.Feature(int64_list=tf.train.Int64List(value=[LP_CLASS])),
        "image/object/difficult": tf.train.Feature(int64_list=tf.train.Int64List(value=[0])),
        "image/object/truncated": tf.train.Feature(int64_list=tf.train.Int64List(value=[0])),
        "image/object/view": tf.train.Feature(bytes_list=tf.train.BytesList(value=["Unspecified".encode("utf-8")])),
    }))
    return example


def create_tfrecord(dataset_path, output_path, samples_per_file):
    print(f"creating {dataset_path} at {output_path}")

    # create the output directory if not exist
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # read the dataset
    with open(dataset_path, "r") as fread:
        lines = fread.read().splitlines()

    # write examples into tfrecord
    fid = 0
    partition_path = output_path + f"-{fid:02d}"
    tfwriter = tf.io.TFRecordWriter(partition_path)
    for i, subpath in enumerate(lines):
        if (i + 1) % samples_per_file == 0:
            fid += 1
            partition_path = output_path + f"-{fid:02d}"
            tfwriter.close()
            tfwriter = tf.io.TFRecordWriter(partition_path)

        filepath = os.path.join(DATASET_DIR, subpath)
        filename = os.path.basename(filepath)
        annotation = filename_to_annotation(filename)
        example = create_example(i + 1, filepath, annotation)
        tfwriter.write(example.SerializeToString())
    tfwriter.close()

In [ ]:
import os
import hashlib
import PIL.Image
import tensorflow as tf
import xml.etree.ElementTree as ET


def parse_xml(annotation_path):
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    bndbox = root.find(".//bndbox")
    obj_name = root.find(".//object/name").text

    annotation = {
        "left": int(bndbox.find("xmin").text),
        "top": int(bndbox.find("ymin").text),
        "right": int(bndbox.find("xmax").text),
        "bottom": int(bndbox.find("ymax").text),
        "class_name": obj_name
    }

    return annotation


def create_example(id, image_path, annotation):
    img_raw = open(image_path, "rb").read()
    key = hashlib.sha256(img_raw).hexdigest()
    filename = os.path.basename(image_path)
    width, height = PIL.Image.open(image_path).size

    example = tf.train.Example(features=tf.train.Features(feature={
        "image/height": tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        "image/width": tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        "image/filename": tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode("utf-8")])),
        "image/source_id": tf.train.Feature(bytes_list=tf.train.BytesList(value=[str(id).encode("utf-8")])),
        "image/key/sha256": tf.train.Feature(bytes_list=tf.train.BytesList(value=[key.encode("utf-8")])),
        "image/encoded": tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
        "image/format": tf.train.Feature(bytes_list=tf.train.BytesList(value=["jpg".encode("utf-8")])),
        "image/object/bbox/xmin": tf.train.Feature(float_list=tf.train.FloatList(value=[annotation["left"] / width])),
        "image/object/bbox/xmax": tf.train.Feature(float_list=tf.train.FloatList(value=[annotation["right"] / width])),
        "image/object/bbox/ymin": tf.train.Feature(float_list=tf.train.FloatList(value=[annotation["top"] / height])),
        "image/object/bbox/ymax": tf.train.Feature(float_list=tf.train.FloatList(value=[annotation["bottom"] / height])),
        "image/object/class/text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[annotation["class_name"].encode("utf-8")])),
        "image/object/class/label": tf.train.Feature(int64_list=tf.train.Int64List(value=[1])),  # You might want to map this to specific labels
        "image/object/difficult": tf.train.Feature(int64_list=tf.train.Int64List(value=[0])),
        "image/object/truncated": tf.train.Feature(int64_list=tf.train.Int64List(value=[0])),
        "image/object/view": tf.train.Feature(bytes_list=tf.train.BytesList(value=["Unspecified".encode("utf-8")])),
    }))
    return example


def create_tfrecord(image_dir, annotation_dir, output_path, subset):
    print(f"Creating TFRecord file for {subset} dataset in {image_dir} at {output_path}")

    # Create the output directory if it doesn't exist
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    files = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]
    partition_path = output_path + f"-{subset}.tfrecord"
    tfwriter = tf.io.TFRecordWriter(partition_path)

    for i, filename in enumerate(files):
        image_path = os.path.join(image_dir, filename)
        annotation_path = os.path.join(annotation_dir, os.path.splitext(filename)[0] + '.xml')

        if not os.path.exists(annotation_path):
            print(f"Annotation file for {filename} does not exist. Skipping.")
            continue

        annotation = parse_xml(annotation_path)
        example = create_example(i + 1, image_path, annotation)
        tfwriter.write(example.SerializeToString())

    tfwriter.close()
    print(f"TFRecord file for {subset} created successfully.")

In [ ]:
import os
import xml.etree.ElementTree as ET
from difflib import get_close_matches

def get_closest_label(class_name, label_map):
    labels = list(label_map.values())
    closest_match = get_close_matches(class_name, labels, n=1)
    return closest_match[0] if closest_match else None

def generate_filename_from_annotation(image_path, annotation_path, label_map):
    tree = ET.parse(annotation_path)
    root = tree.getroot()

    filename = os.path.basename(image_path)
    image_size = root.find("size")
    width = int(image_size.find("width").text)
    height = int(image_size.find("height").text)
    depth = int(image_size.find("depth").text)

    objects = root.findall("object")

    area_ratio = 1  # Placeholder, calculate if needed
    tilt_h = 0  # Placeholder, calculate if needed
    tilt_v = 0  # Placeholder, calculate if needed

    bbox_str = ""
    vertices_str = ""
    class_ids = []
    for obj in objects:
        bndbox = obj.find("bndbox")
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)

        class_name = obj.find("name").text
        closest_label = get_closest_label(class_name, label_map)
        if closest_label:
            class_id = list(label_map.keys())[list(label_map.values()).index(closest_label)]
            class_ids.append(class_id)
        else:
            continue

        bbox_str += f"{xmin}&{ymin}_{xmax}&{ymax}-"
        vertices_str += f"{xmax}&{ymax}_{xmin}&{ymax}_{xmin}&{ymin}_{xmax}&{ymin}-"

    if not class_ids:
        return None

    class_ids_str = "_".join(map(str, class_ids))
    brightness = 0  # Placeholder, calculate if needed
    blurriness = 0  # Placeholder, calculate if needed

    annotation_str = f"{area_ratio}-{tilt_h}_{tilt_v}-{bbox_str[:-1]}-{vertices_str[:-1]}-{class_ids_str}-{brightness}-{blurriness}.jpg"

    return annotation_str

def generate_annotation_file(train_dir, output_file, label_map):
    images_dir = os.path.join(train_dir, 'images')
    annotations_dir = os.path.join(train_dir, 'annotation')

    with open(output_file, 'w') as f:
        for image_filename in os.listdir(images_dir):
            if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
                image_path = os.path.join(images_dir, image_filename)
                annotation_filename = os.path.splitext(image_filename)[0] + '.xml'
                annotation_path = os.path.join(annotations_dir, annotation_filename)

                if os.path.exists(annotation_path):
                    annotation_str = generate_filename_from_annotation(image_path, annotation_path, label_map)
                    if annotation_str:
                        f.write(f"{annotation_str}\n")

    print(f"Annotation file created successfully at {output_file}")

# Define your dataset path and output file path
TRAIN_DIR = "/content/gdrive/MyDrive/Dataset- -Conference/train"  # Update this path
OUTPUT_FILE = "/content/gdrive/MyDrive/Dataset- -Conference/train/train.txt"  # Update this path

labels = [
    "SpeedLimit60kmh",
    "School",
    "PEDESTRIANCROSSING",
    "SharpBendToTheLeft",
    "UTurn",
    "speedlimit80kmh",
    "NoOvertaking",
    "SharpBendToTheRight",
    "SideRoadLeft",
    "SideRoadRight",
    "speedlimit40kmh",
    "NarrowBridge",
    "RoadHump",
    "nouseofhorn",
    "Crossroads"
]

label_map = {i + 1: label for i, label in enumerate(labels)}

generate_annotation_file(TRAIN_DIR, OUTPUT_FILE, label_map)

Annotation file created successfully at /content/gdrive/MyDrive/Dataset- -Conference/train/train.txt


In [ ]:
create_tfrecord(TRAIN_SET_FILE, os.path.join(TFRECORD_DIR, "train.tfrecord"), SAMPLES_PER_FILE)
# due to the storage limitation on Colab, we skip the testing set
#create_tfrecord(TEST_SET_FILE, os.path.join(TFRECORD_DIR, "test.tfrecord"), SAMPLES_PER_FILE)

creating /content/gdrive/MyDrive/Dataset- -Conference/train/train.txt at /content/gdrive/MyDrive/Dataset- -Conference/tfrecords/train.tfrecord


ValueError: the filename cannot be resolved

In [ ]:
import os
import tensorflow as tf
import xml.etree.ElementTree as ET
from PIL import Image
import io
import difflib

# Define class names and their corresponding IDs
CLASS_NAMES = [
    'SpeedLimit60kmh', 'School', 'PEDESTRIANCROSSING', 'SharpBendToTheLeft',
    'UTurn', 'speedlimit80kmh', 'NoOvertaking', 'SharpBendToTheRight',
    'SideRoadLeft', 'SideRoadRight', 'speedlimit40kmh', 'NarrowBridge',
    'RoadHump', 'nouseofhorn', 'Crossroads'
]

CLASS_DICT = {name: idx for idx, name in enumerate(CLASS_NAMES)}

# Helper function to find the closest class name
def closest_class_name(class_name):
    closest_match = difflib.get_close_matches(class_name, CLASS_DICT.keys(), n=1, cutoff=0.6)
    return closest_match[0] if closest_match else None

# Helper function to create TFRecords
def create_tf_example(img_path, annotation_path):
    # Read the image
    with tf.io.gfile.GFile(img_path, 'rb') as fid:
        encoded_image = fid.read()

    image = Image.open(io.BytesIO(encoded_image))
    width, height = image.size

    # Parse XML annotation
    tree = ET.parse(annotation_path)
    root = tree.getroot()

    filename = root.find('filename').text
    xmin_list = []
    ymin_list = []
    xmax_list = []
    ymax_list = []
    classes_text = []
    classes = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        closest_name = closest_class_name(class_name)
        if closest_name is None:
            continue

        classes_text.append(closest_name.encode('utf8'))
        classes.append(CLASS_DICT[closest_name])

        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        xmin_list.append(float(xmin) / width)
        ymin_list.append(float(ymin) / height)
        xmax_list.append(float(xmax) / width)
        ymax_list.append(float(ymax) / height)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode('utf8')])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode('utf8')])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_image])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'png'])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmin_list)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymin_list)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmax_list)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymax_list)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
    }))

    return tf_example

def create_tfrecords(train_dir, output_path):
    writer = tf.io.TFRecordWriter(output_path)

    image_dir = os.path.join(train_dir, 'images')
    annotation_dir = os.path.join(train_dir, 'annotation')

    for filename in os.listdir(image_dir):
        if filename.endswith('.png'):
            img_path = os.path.join(image_dir, filename)
            annotation_path = os.path.join(annotation_dir, filename.replace('.png', '.xml'))

            if not os.path.exists(annotation_path):
                continue

            tf_example = create_tf_example(img_path, annotation_path)
            writer.write(tf_example.SerializeToString())

    writer.close()

# Run the function
train_dir = '/content/gdrive/MyDrive/Dataset- -Conference/val'
output_path = '/content/gdrive/MyDrive/Dataset- -Conference/val/val.tfrecord'
create_tfrecords(train_dir, output_path)

In [ ]:
import os

def convert_dataset_to_file_list(dataset_dir, output_file):
    with open(output_file, 'w') as f:
        for subset in ['train', 'val', 'test']:
            image_dir = os.path.join(dataset_dir, subset, 'images')
            annotation_dir = os.path.join(dataset_dir, subset, 'annotation')

            if not os.path.exists(image_dir) or not os.path.exists(annotation_dir):
                print(f"Subset directory {subset} does not exist or is incomplete.")
                continue

            files = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]

            for filename in files:
                image_path = os.path.join(subset, 'images', filename)
                annotation_path = os.path.join(subset, 'annotation', os.path.splitext(filename)[0] + '.xml')

                if not os.path.exists(os.path.join(dataset_dir, annotation_path)):
                    print(f"Annotation file for {filename} does not exist. Skipping.")
                    continue

                f.write(f"{image_path}\n")

    print(f"File list created successfully at {output_file}")

# Define your dataset path and output file path
%mkdir tfrecords
DATASET_DIR = "/content/gdrive/MyDrive/Dataset- -Conference"  # Update this path
OUTPUT_FILE = "/content/tfrecords/train.txt"  # Update this path

convert_dataset_to_file_list(DATASET_DIR, OUTPUT_FILE)

In [ ]:
import tensorflow as tf
for example in tf.python_io.tf_record_iterator("/content/gdrive/MyDrive/Dataset- -Conference/val/val.tfrecord"):
  print(tf.train.Example.FromString(example))

AttributeError: module 'tensorflow' has no attribute 'python_io'

# Model training

## Create the config YAML file

In [ ]:
CONFIG_DIR = os.path.join(PROJ_DIR, "configs")
CONFIG_FILE = os.path.join(CONFIG_DIR, "default.yaml")
if not os.path.exists(CONFIG_DIR):
    os.mkdir(CONFIG_DIR)


labels = [
    "SpeedLimit60kmh",
    "School",
    "PEDESTRIANCROSSING",
    "SharpBendToTheLeft",
    "UTurn",
    "speedlimit80kmh",
    "NoOvertaking",
    "SharpBendToTheRight",
    "SideRoadLeft",
    "SideRoadRight",
    "speedlimit40kmh",
    "NarrowBridge",
    "RoadHump",
    "nouseofhorn",
    "Crossroads"
]


label_map = {i + 1: label for i, label in enumerate(labels)}

label_map_str = ", ".join([f"{k}: {v}" for k, v in label_map.items()])

config_text = f"""
image_size: 1024x1024
num_classes: {len(labels)}
label_map: {{{label_map_str}}}
input_rand_hflip: true
jitter_min: 0.8
jitter_max: 1.2
"""

with open(CONFIG_FILE, "w") as fwrite:
    fwrite.write(config_text)

## Training parameters

In [ ]:
CKPT = MODEL

#TRAIN_SET = os.path.join(TFRECORD_DIR, "train.tfrecord-*")
#VAL_SET = os.path.join(TFRECORD_DIR, "val.tfrecord-*")
TRAIN_SET = "/content/gdrive/MyDrive/Dataset- -Conference/tfrecords/train.tfrecord-train.tfrecord"
VAL_SET = "/content/gdrive/MyDrive/Dataset- -Conference/tfrecords/val.tfrecord-val.tfrecord"
MODEL_DIR_TMP = os.path.join(PROJ_DIR, "tmp", f"{MODEL}-finetune")
TRAIN_NUM_EXAMPLES = 5000
EVAL_NUM_EXAMPLES = 5000
EPOCHS = 1
BATCH_SIZE = 2

## Remove the previous checkpoint (if exists) before restarting the training

In [ ]:
if os.path.exists(MODEL_DIR_TMP):
    !rm -rf {MODEL_DIR_TMP}

## Start training

In [ ]:
TRAIN_SET = "/content/gdrive/MyDrive/Dataset- -Conference/train/train.tfrecord"
VAL_SET = "/content/gdrive/MyDrive/Dataset- -Conferenc/val/val.tfrecord"

In [ ]:
!python /content/automl/efficientdet/main.py \
    --mode=train_and_eval \
    --train_file_pattern={TRAIN_SET} \
    --val_file_pattern={VAL_SET} \
    --model_name={MODEL} \
    --model_dir={MODEL_DIR_TMP} \
    --ckpt={CKPT} \
    --train_batch_size={BATCH_SIZE} \
    --eval_batch_size={BATCH_SIZE} \
    --num_epochs={EPOCHS} \
    --num_examples_per_epoch={TRAIN_NUM_EXAMPLES} \
    --eval_samples={EVAL_NUM_EXAMPLES} \
    --hparams={CONFIG_FILE}

2024-07-14 16:16:15.220913: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 16:16:15.221152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 16:16:15.224729: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-14 16:16:16.753057: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
FATAL Flags parsing error: Unknown command line flag 'Conference/train/train.tfrecord'
Pass --helpshort or --helpfull to see help on flags.


# Model inference

## Export the model (from checkpoint to saved model format)

In [ ]:
MODEL_DIR_EXPORT = os.path.join(PROJ_DIR, "models", f"{MODEL}-finetune")

MIN_SCORE_THRESHOLD = 0.1

!python /content/automl/efficientdet/main_inspect.py \
    --runmode=saved_model \
    --model_name={MODEL} \
    --ckpt_path={MODEL_DIR_TMP} \
    --saved_model_dir={MODEL_DIR_EXPORT} \
    --min_score_thresh={MIN_SCORE_THRESHOLD} \
    --hparams={CONFIG_FILE}

In [ ]:
!python model_inspect.py --runmode=infer \
  --model_name=efficientdet-d0   --ckpt_path=efficientdet-d0 \
  --hparams=voc_config.yaml  \
  --input_image=img.png --output_image_dir=/tmp/

## Run inference

In [ ]:
TEST_IMAGE_DIR = os.path.join(DATASET_DIR, "ccpd_base")
TEST_IMAGE_FILES = os.path.join(TEST_IMAGE_DIR, "*.jpg")
#TEST_IMAGE_FILES = os.path.join(TEST_IMAGE_DIR, "09-1_13-81&477_603&621-578&604_81&621_106&494_603&477-0_0_24_31_3_27_30-102-134.jpg")

RESULT_DIR = os.path.join(PROJ_DIR, "results")
if not os.path.exists(RESULT_DIR):
    os.mkdir(RESULT_DIR)

# we can set a higher threshold here
# however, the results are already filtered with the previous threshold during the model exporting
MIN_SCORE_THRESHOLD = 0.5

!python /content/automl/efficientdet/main_inspect.py \
    --runmode=saved_model_infer \
    --saved_model_dir={MODEL_DIR_EXPORT} \
    --model_name={MODEL} \
    --input_image={TEST_IMAGE_FILES} \
    --output_image_dir={RESULT_DIR} \
    --min_score_thresh={MIN_SCORE_THRESHOLD} \
    --hparams={CONFIG_FILE}

## Visualize the results

In [ ]:
from IPython.display import Image, display

MAX_SHOW_RESULTS = 10

for i in range(MAX_SHOW_RESULTS):
    fname = os.path.join(RESULT_DIR, "{}.jpg".format(i))
    if os.path.exists(fname):
        print(fname)
        display(Image(fname))